In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import warnings
import csv
import time

warnings.filterwarnings('ignore')

## The following code scapes weather data from wunderground.com

In [4]:
root_url = "https://www.wunderground.com/history/airport/KNYC/2015/"
second_part_url = "/DailyHistory.html?req_city=New%20York&req_state=NY&req_statename=New%20York&reqdb.zip=10001&reqdb.magic=1&reqdb.wmo=99999&MR=1"

In [5]:
## Put all the dates of the year in a list
month = list(range(1,13))
days = list(range(1,32))

days_of_year = []

for m in month:
    for d in days:
        days_of_year.append(str(m) + "/" + str(d))

In [7]:
##Remove invalid days
invalid_days = ["2/29",'2/30','2/31','4/31','6/31','9/31','11/31']
indexes_to_remove = []
for i in invalid_days:
    indexes_to_remove.append(days_of_year.index(i))

In [10]:
for i in indexes_to_remove[::-1]:
    days_of_year.pop(i)

In [12]:
links_to_scrape = []
for i in days_of_year:
    links_to_scrape.append(root_url + i + second_part_url)

In [117]:
## Function to collect data from the web
def get_weather(url, date):
    
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page)
    
    address = soup.find(text="Mean Temperature")
    b_tag = address.parent
    td_tag = b_tag.parent
    next_td_tag = td_tag.findNext('td')
    mean_temperature = next_td_tag.findNext("span").findNext("span").contents
    
    address = soup.find(text="Max Temperature")
    b_tag = address.parent
    td_tag = b_tag.parent
    next_td_tag = td_tag.findNext('td')
    max_temperature = next_td_tag.findNext("span").findNext("span").contents
    
    address = soup.find(text="Min Temperature")
    b_tag = address.parent
    td_tag = b_tag.parent
    next_td_tag = td_tag.findNext('td')
    min_temperature = next_td_tag.findNext("span").findNext("span").contents
    
    address = soup.find(text="Average Humidity")
    b_tag = address.parent
    td_tag = b_tag.parent
    next_td_tag = td_tag.findNext('td')
    average_humidity = next_td_tag.contents
    
    address = soup.find(text="Maximum Humidity")
    b_tag = address.parent
    td_tag = b_tag.parent
    next_td_tag = td_tag.findNext('td')
    max_humidity = next_td_tag.contents
    
    address = soup.find(text="Minimum Humidity")
    b_tag = address.parent
    td_tag = b_tag.parent
    next_td_tag = td_tag.findNext('td')
    min_humidity = next_td_tag.contents
    
    return [date + '/2015', int(mean_temperature[0]), int(max_temperature[0]), int(min_temperature[0]), int(average_humidity[0]), int(max_humidity[0]), int(min_humidity[0])]

In [149]:
##Scraping data for all days of 2015 and then storing it in a pandas dataframe
weather = pd.DataFrame(columns=['Date', 'mean_temp', "max_temp", "min_temp", "average_humid", "max_humid", "min_humid"])
count = 0
for link, date in zip(links_to_scrape, days_of_year):
    x = get_weather(link, date)
    weather = weather.append(pd.Series(x, index=['Date', 'mean_temp', "max_temp", "min_temp", "average_humid", "max_humid", "min_humid"]), ignore_index=True)
    count += 1
    if count % 50 == 0:
        print (count)

50
100
150
200
250
300
350


In [151]:
weather.head()

,Date,mean_temp,max_temp,min_temp,average_humid,max_humid,min_humid
0,1/1/2015,33.0,39.0,27.0,37.0,46.0,27.0
1,1/2/2015,39.0,42.0,35.0,43.0,52.0,33.0
2,1/3/2015,38.0,42.0,33.0,68.0,92.0,44.0
3,1/4/2015,49.0,56.0,41.0,88.0,93.0,83.0
4,1/5/2015,35.0,49.0,21.0,38.0,49.0,26.0


In [152]:
weather.to_csv("weather_data.csv")